In [1]:
import sys
sys.path.append('../Data')
sys.path.append('../GenerateLabel')

import pandas as pd
import numpy as np
import time
from CommentDataset import CommentDataset
from PageRank import WordGraph, TopicalPageRank

In [2]:
comment_list_name = "AIhyouka_sannkou_gyakutaimiokuri"
comment_list_path = "2023_07_11/" + comment_list_name + ".csv"

model_path = "model/pickle/2023_07_11/AIhyouka_sannkou_gyakutaimiokuri_default.pickle"

In [3]:
# 指定すべき文字コードはファイルによる ex.cp932, utf-8
df = pd.read_csv("../Data/yahoo/"+comment_list_path, encoding="utf-8", header=None)
comment_list = df[0][1:].to_numpy().tolist()
comment_dataset = CommentDataset(comment_list)
comment_dataset.delete_comment_specified_string(50)
comment_dataset.formatted_input_hlda(appear_tagging_list=["名詞", "形容詞", "動詞"], stop_tagging_list=["助詞", "助動詞", "非自立"])

In [4]:
from model.HldaModel import ExpandHldaModel

expandHlda = ExpandHldaModel(pickle_path=model_path)

In [5]:
print(expandHlda.hlda.document_leaves[0].node_id)

971


In [6]:
expandHlda.print_nodes(n_words=5, weights=True)

topic=0 level=0 (documents=1200): 保護 (590), 判断 (584), 思う (560), れる (427), 人間 (358), 
    topic=1 level=1 (documents=250): 児相 (96), 虐待 (70), 子供 (51), 職員 (46), 対応 (32), 
        topic=2 level=2 (documents=71): 保育園 (61), 長期 (44), 欠席 (35), 問題 (31), 面会 (30), 
        topic=3 level=2 (documents=7): 奪う (4), 技術 (3), 機運 (2), 極み (2), 遭難 (2), 
        topic=13 level=2 (documents=24): やつ (7), 足る (4), 知識 (4), 生活 (4), そもそも (3), 
        topic=484 level=2 (documents=6): 投資 (4), 技術 (3), 児童福祉司 (2), ご冥福をお祈りします (2), 福祉 (2), 
        topic=578 level=2 (documents=11): 説明 (4), 怒る (3), 感覚 (2), そうだ (2), 参考 (2), 
        topic=807 level=2 (documents=5): 処罰 (3), はかる (2), バス (2), 悔いる (2), 狡猾 (2), 
        topic=867 level=2 (documents=15): 一般 (4), 上司 (4), 専門知識 (3), 表示 (3), 対策 (3), 
        topic=892 level=2 (documents=3): 強化 (2), 他人事 (2), 周囲 (1), 緩和 (1), 与する (1), 
        topic=894 level=2 (documents=7): 解釈 (2), 正義感 (1), 確かめる (1), 聞く (1), コンピューター (1), 
        topic=980 level=2 (documents=4): 実務 (2), 逃れ (2), 構造 (

In [7]:
# 指定しているトピックのキーフレーズ抽出
topic_id = 1619
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)

if topic_document_list:
    tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
    topic_word_weighted = expandHlda.get_weighted(topic_id)
    phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)
    
    for phrase in phrase_list[0:10]:
        print(phrase)

1619がみつかりません


In [8]:
topic_id = 1543
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)
if topic_document_list:
    tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
    topic_word_weighted = expandHlda.get_weighted(topic_id)
    phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)
    
    for phrase in phrase_list[0:10]:
        print(phrase)

1543がみつかりません


In [9]:
topic_id = 13
topic_document_list = expandHlda.get_topic_document(comment_list=comment_list, corpus=comment_dataset.comment_list, topic_id=topic_id)
if topic_document_list:
    tpr = TopicalPageRank(collection=topic_document_list, appear_tagging_list=["名詞", "形容詞"], w=10)
    topic_word_weighted = expandHlda.get_weighted(topic_id)
    phrase_list = tpr.extract_phrase(damping_factor=0.3, word_weighted_list=topic_word_weighted)
    
    for phrase in phrase_list[0:10]:
        print(phrase)

['生活', ['生活'], 0.05797057553408798]
['机上', ['机上'], 0.0375]
['一緒', ['一緒'], 0.031018477956193075]
['虐待知識', ['虐待', '知識'], 0.03000973329162732]
['知識量', ['知識', '量'], 0.02839509103148987]
['識者', ['識者'], 0.027824005333764723]
['庶民', ['庶民'], 0.027428124999999998]
['担当職員', ['担当', '職員'], 0.020355099110463626]
['ニュース', ['ニュース'], 0.01640990558154682]
['行政', ['行政'], 0.014338888097505198]


In [10]:
expandHlda.print_phrases(comment_list=comment_list, corpus=comment_dataset.comment_list, n_phrase=3, with_score=True)

topic=0 level=0 (documents=1200): 判断 (0.066),保護判断 (0.065),保護 (0.064),
    topic=1 level=1 (documents=250): 虐待 (0.063),子供 (0.048),職員 (0.043),
        topic=2 level=2 (documents=71): 保育園 (0.094),長期 (0.076),長期欠席 (0.069),
        topic=3 level=2 (documents=7): 政府 (0.055),世界一隠ぺい (0.050),世界一平和 (0.049),
        topic=13 level=2 (documents=24): 生活 (0.058),机上 (0.037),一緒 (0.031),
        topic=484 level=2 (documents=6): 投資 (0.084),利益 (0.042),教育福祉行政 (0.042),
        topic=578 level=2 (documents=11): 説明 (0.130),介護 (0.064),安全 (0.037),
        topic=807 level=2 (documents=5): 面会 (0.111),裁判所管轄 (0.056),保留 (0.052),
        topic=867 level=2 (documents=15): 上司 (0.046),対策 (0.034),割合 (0.033),
        topic=892 level=2 (documents=3): 周囲 (0.049),権限強化 (0.049),システム (0.049),
        topic=894 level=2 (documents=7): 解釈 (0.117),コンピューター頼り (0.071),業界 (0.061),
        topic=980 level=2 (documents=4): 現場 (0.065),介護 (0.046),賃金 (0.046),
        topic=1080 level=2 (documents=3): 端々 (0.122),仕草 (0.116),態度 (0.112),
      

C:\Users\tatsu\OneDrive\デスクトップ\program\TechnicalInvestigation\HldaInput\../GenerateLabel\PageRank.py:105: RuntimeWarning: invalid value encountered in double_scalars
  word_probability = new_word_weighted_list[word_score_i_index] / sum_word_weighted


        topic=1102 level=2 (documents=3): イジメ (0.000),虐待 (0.000),人間 (0.000),
        topic=1127 level=2 (documents=8): 都道府県 (0.191),市町村 (0.089),先々 (0.088),
        topic=1140 level=2 (documents=9): データ (0.078),AI云々 (0.041),余程効果的 (0.029),
        topic=1143 level=2 (documents=4): 大人 (0.092),予算 (0.087),隠蔽要素 (0.064),
        topic=1162 level=2 (documents=8): 分母 (0.086),アナログ (0.047),統計 (0.046),
        topic=1175 level=2 (documents=4): 現状 (0.154),詳細 (0.140),公安並み (0.082),
        topic=1190 level=2 (documents=6): 倫理 (0.055),偏り (0.052),プライバシー配慮 (0.051),
        topic=1195 level=2 (documents=6): 注意 (0.106),殺人事件 (0.063),事件性 (0.057),
        topic=1199 level=2 (documents=4): 黒人 (0.134),収入 (0.068),追求 (0.034),
        topic=1214 level=2 (documents=12): 人権 (0.041),税金 (0.040),作業 (0.023),
        topic=1222 level=2 (documents=1): 一定 (0.251),有限 (0.248),基準 (0.055),
    topic=15 level=1 (documents=195): 職員 (0.052),責任 (0.047),虐待 (0.047),
        topic=19 level=2 (documents=8): 真面目で優しい職員 (0.077),給料ボーナス (